# Dataset description
* 32 participants [s01 to s32]
* 40 pieces of music
* 40 channels out of which the first 32 are relevant 
* 8064 readings recorded at 128Hz


In [1]:
from deap_pre_process import *

In [2]:
eeg1d,eeg2d,val,aro = get_subject_data(subject_no=1,media_no=1)

[+] Reading subject: s01 Media: 1


In [3]:
len(eeg2d.reshape(-1,9,9,1))

7680

In [8]:
eeg2d.shape

(7680, 9, 9)

In [4]:
val,aro

(1, 1)

# Making the dummy network


In [10]:
from rnn_function import *
from cnn_function import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [31]:
#Constant :
WINDOW_SIZE = 64
#For rnn layers
INPUT_NODES = 32
HIDDEN_ST = 32
HOLD_PROBA = 0.5
RNN_INPUT_SIZE = 32

#For cnn layers
CONV_IP_SHAPE = [-1,9,9,1]
CONV1_KERNEL = [4,4,1,32]
CONV2_KERNEL = [4,4,32,64]
CONV3_KERNEL = [4,4,64,128]
REDUCTION_KERNEL = [1,1,128*WINDOW_SIZE,13]

#For dnn layers (for the paper code)
N_FC_IN = 1024
N_FC_OUT= 1024

#FOR PREDICTIONS
N_LABELS = 2
LEARNING_RATE = 0.001

In [27]:
#FUNCTION FORM https://github.com/ynulonger/ijcnn/blob/master/cv.py
#FOR SHAPING THE INPUT OF LSTM
def apply_fully_connect(x, x_size, fc_size):
    fc_weight = init_weight([x_size, fc_size])
    fc_bias = init_bias([fc_size])
    return tf.nn.elu(tf.add(tf.matmul(x, fc_weight), fc_bias))

#FOR READOUT OF THE FINAL LAYER
def apply_readout(x, x_size, readout_size):
    readout_weight = init_weight([x_size, readout_size])
    readout_bias = init_bias([readout_size])
    return tf.add(tf.matmul(x, readout_weight), readout_bias)

In [20]:
#Resetting tf default graph
tf.reset_default_graph()

#CNN placeholders and model variables
x_data_cnn = tf.placeholder(dtype = tf.float32,shape=(None,9,9),name='cnn_data')
x_image = tf.reshape(x_data_cnn,CONV_IP_SHAPE,name='cnn_image')

#LAYERS
conv1 = conv_layer(x_image,CONV1_KERNEL,name="convo_l1")
conv2 = conv_layer(conv1,CONV2_KERNEL,name="convo_l2")
conv3 = conv_layer(conv2,CONV3_KERNEL,name="convo_l3")

#Depth variable
depth = tf.reshape(conv3,[-1,9,9,128* WINDOW_SIZE])

#Size reduction conv layer
reduced_data = conv_layer(depth,REDUCTION_KERNEL,name="conv_reduced")

#Flattening convolution layer
shape = reduced_data.get_shape().as_list()
final_flat = tf.reshape(reduced_data, [-1, shape[1] * shape[2] * shape[3]])
cnn_out_fuse = final_flat

#RNN placeholder and model variables
x_data_rnn = tf.placeholder(dtype = tf.float32,shape=(None,WINDOW_SIZE,INPUT_NODES),name='rnn_data') #TIME_STEP = WINDOW_SIZE
keep_prob = tf.placeholder(dtype=tf.float32)

#Shaping the RNN input
shape = x_data_rnn.get_shape().as_list()
rnn_in_flat = tf.reshape(x_data_rnn, [-1, shape[2]]) #[batch_size*n_time_step, n_electrode]
rnn_fc_in = apply_fully_connect(rnn_in_flat, shape[2], N_FC_IN) #[batch_size*n_time_step, n_electrode]
lstm_in = tf.reshape(rnn_fc_in, [-1,WINDOW_SIZE,N_FC_IN]) #[batch_size, n_time_step, n_fc_in]

#Making cells
cell1 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cell2 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cells = [cell1,cell2]
final_cell = create_Stacked_cell(cells)

#Laying out the network
output, states = create_RNN(final_cell,lstm_in)

#Unstacking the output
output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')

#Selecting the final output form the layer
rnn_output = output[-1] #[batch, fc_size]

#Shaping the output
lstm_fc_out = dnn_layer(rnn_output,N_FC_OUT) #[batch_size, n_fc_out]
lstm_fc_drop = tf.nn.dropout(lstm_fc_out, keep_prob)

#Placeholder for true values
y_true = tf.placeholder(dtype=tf.float32,shape=(None,N_LABELS)) #[Valance, Arousal]

In [23]:
#Flattening the layer
final_connected_layer = tf.concat([cnn_out_fuse, lstm_fc_drop], axis=1)
shape = final_connected_layer.get_shape().as_list()
print("\nfuse_cnn_rnn:", shape)


fuse_cnn_rnn: [None, 2077]


In [32]:
#Creating output variables 
y_ = apply_readout(final_connected_layer, shape[1], N_LABELS)
y_pred = tf.argmax(tf.nn.softmax(y_), 1, name="y_pred")
y_posi = tf.nn.softmax(y_, name="y_posi")

#Making cost functions
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y_true), name='loss')
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

In [33]:
# Saver and variable initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# REMAINING:
* Session

In [35]:
eeg2d.shape

(7680, 9, 9)

In [37]:
eeg1d.shape

(7680, 32)

In [45]:
with tf.Session() as sess:
    sess.run(init)
    eeg1d,eeg2d,val,aro = get_subject_data(subject_no=1,media_no=1)
    for i in range(0,7680,WINDOW_SIZE):
        print("WINDOW FROM",i," TO ",i+WINDOW_SIZE)
        pred = np.array([val,aro]).reshape(1,2)
        x_rnn = eeg1d[i:i+WINDOW_SIZE].reshape(-1,WINDOW_SIZE,32)
        x_cnn = eeg2d[i:i+WINDOW_SIZE]
        
        sess.run(train, feed_dict = {keep_prob:0.5,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: pred})
    
    
    
    

[+] Reading subject: s01 Media: 1
WINDOW FROM 0  TO  64
WINDOW FROM 64  TO  128
WINDOW FROM 128  TO  192
WINDOW FROM 192  TO  256
WINDOW FROM 256  TO  320
WINDOW FROM 320  TO  384
WINDOW FROM 384  TO  448
WINDOW FROM 448  TO  512
WINDOW FROM 512  TO  576
WINDOW FROM 576  TO  640
WINDOW FROM 640  TO  704
WINDOW FROM 704  TO  768
WINDOW FROM 768  TO  832
WINDOW FROM 832  TO  896
WINDOW FROM 896  TO  960
WINDOW FROM 960  TO  1024
WINDOW FROM 1024  TO  1088
WINDOW FROM 1088  TO  1152
WINDOW FROM 1152  TO  1216
WINDOW FROM 1216  TO  1280
WINDOW FROM 1280  TO  1344
WINDOW FROM 1344  TO  1408
WINDOW FROM 1408  TO  1472
WINDOW FROM 1472  TO  1536
WINDOW FROM 1536  TO  1600
WINDOW FROM 1600  TO  1664
WINDOW FROM 1664  TO  1728
WINDOW FROM 1728  TO  1792
WINDOW FROM 1792  TO  1856
WINDOW FROM 1856  TO  1920
WINDOW FROM 1920  TO  1984
WINDOW FROM 1984  TO  2048
WINDOW FROM 2048  TO  2112
WINDOW FROM 2112  TO  2176
WINDOW FROM 2176  TO  2240
WINDOW FROM 2240  TO  2304
WINDOW FROM 2304  TO  2368
WI